<a href="https://colab.research.google.com/github/qinalan10/cs598-patient-phenotyping/blob/master/Code/Data_Pre_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Preprocessing - Preparing the file

We have 2 files - a notes data file and annotations data. We need to join the files together and create a final file for processing.

1.   Read the notes file.
2.   Read annotations file.
3.   Join the notes file and the annotations file by hospital admission id.
4.   Keep the relevant fields for our operation and drop the rest.
5.   Store the final final file.



Importing the required libraries -

In [1]:
# importing libaries
from google.colab import files
from google.colab import drive
import pandas as pd 
import torch
from torch.utils.data import Dataset
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
import re 
import numpy as np 

from torch.utils.data import Dataset, DataLoader

from nltk.corpus import stopwords

drive.mount('/content/drive')

Mounted at /content/drive


Setting up the file path and reading the files

In [20]:
data_path = '/content/drive/MyDrive/Data/'
notes_path = data_path + 'NOTEEVENTS.csv'
notes_df = pd.read_csv(notes_path)
annotations_path = data_path + 'annotations.csv'
annotations = pd.read_csv(annotations_path)

<ipython-input-20-3421aeb6aca6>:3: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  notes_df = pd.read_csv(notes_path)


Quick look at the data.

In [21]:
notes_df.head(1)

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
0,174,22532,167853.0,2151-08-04,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2151-7-16**] Dischar...


In [22]:
annotations.head(1)

,Hospital.Admission.ID,subject.id,chart.time,cohort,Obesity,Non.Adherence,Developmental.Delay.Retardation,Advanced.Heart.Disease,Advanced.Lung.Disease,Schizophrenia.and.other.Psychiatric.Disorders,Alcohol.Abuse,Other.Substance.Abuse,Chronic.Pain.Fibromyalgia,Chronic.Neurological.Dystrophies,Advanced.Cancer,Depression,Dementia,Unsure
0,118003,3644,118003,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0


In [23]:
hamd = pd.merge(annotations, notes_df, how = 'inner', left_on = ['Hospital.Admission.ID', 'subject.id'], right_on = ['HADM_ID', 'SUBJECT_ID'])
discharges = hamd[(hamd.CATEGORY == 'Discharge summary') & (hamd.DESCRIPTION == 'Report')]
discharges = hamd[(hamd.CATEGORY == 'Discharge summary') & (hamd.DESCRIPTION == 'Report')]#['DESCRIPTION'].unique()
discharges.head(2)
x = discharges[['Obesity', 'Non.Adherence', 'Developmental.Delay.Retardation',
        'Advanced.Heart.Disease',  'Advanced.Lung.Disease',
        'Schizophrenia.and.other.Psychiatric.Disorders', 'Alcohol.Abuse',
        'Other.Substance.Abuse', 'Chronic.Pain.Fibromyalgia',
        'Chronic.Neurological.Dystrophies', 'Advanced.Cancer', 'Depression',
        'Dementia', 'Unsure', 'TEXT']]
x.drop_duplicates(inplace = True)
x.drop_duplicates(subset = ['TEXT'], keep = 'first', inplace = True)

<ipython-input-23-f04f5ecaedbc>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x.drop_duplicates(inplace = True)
<ipython-input-23-f04f5ecaedbc>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x.drop_duplicates(subset = ['TEXT'], keep = 'first', inplace = True)


In [24]:
final_file_path = data_path + 'final_file.csv'
x.to_csv(final_file_path)

Read the final file and look at the notes data for few patients.

In [27]:
final_file_df = pd.read_csv(final_file_path)
final_file_df.drop('Unnamed: 0', axis=1, inplace=True)
final_file_df.head()


"Admission Date:  [**2200-4-7**]     Discharge Date:  [**2200-4-10**]\n\nDate of Birth:   [**2146-9-21**]     Sex:  F\n\nService:  CARDIAC INTENSIVE CARE MEDICINE\n\nCHIEF COMPLAINT:  The patient was admitted to the Cardiac\nIntensive Care Unit Medicine Service on [**2200-4-7**], with the\nchief complaint of acute myocardial infarction and fever.\n\nHISTORY OF PRESENT ILLNESS:  The patient is a 53 year old\nwhite female with a history of coronary artery disease,\nhypertension, hypercholesterolemia and two pack per day\ntobacco use with previous coronary artery bypass graft\nsurgery presenting to an outside hospital on [**2200-4-6**], with a\ntwo day history of fevers and confusion.  The patient had a\nCT scan of the chest at that time which revealed pneumonia by\nreport in the left lower lobe.\n\nWhile in the outside hospital Emergency Department, the\npatient complained of chest pain.  The patient states that\nshe has had this pain for approximately two weeks with no\nrelief.  She was

In [29]:
final_file_df["TEXT"][1]

"Admission Date:  [**2107-2-13**]              Discharge Date:   [**2107-2-15**]\n\nDate of Birth:  [**2076-8-17**]             Sex:   M\n\nService: MEDICINE\n\nAllergies:\nceclor / compazine\n\nAttending:[**Known firstname 2751**]\nChief Complaint:\nAbdominal pain\n\nMajor Surgical or Invasive Procedure:\nNone\n\nHistory of Present Illness:\nThis is a 30 year old male with PMH of barrett's esophagous s/p\nnissen fundoplication in [**2093**], emergent splenectomy in [**2099**] s/p\nsnow boarding accident, metastatic soft tissue fibrosarcoma s/p\nsmall bowel resection in [**2104**] at [**Hospital1 2025**], started chemo in [**Month (only) **]\nwith transition to temador chemotherapy (last dose during first\nweek of [**Month (only) **]), who presents with fevers to 102, crampy\nabdominal pain, and watery, nonbloody diarrhea.\n.\nTwo-three weeks prior to admission, patient originally admitted\nto [**Hospital1 18**] [**Location (un) 620**] with fevers, abdominal pain, and watery\ndiarrhea.

In [31]:
final_file_df["TEXT"][100]

"Admission Date:  [**2133-2-16**]              Discharge Date:   [**2133-2-26**]\n\nDate of Birth:  [**2092-7-8**]             Sex:   M\n\nService: MEDICINE\n\nAllergies:\nPatient recorded as having No Known Allergies to Drugs\n\nAttending:[**First Name3 (LF) 338**]\nChief Complaint:\nRespiratory failure\n\nMajor Surgical or Invasive Procedure:\nChest Tube Placement\nCentral Line Placement\nPICC Line placement\n\n\nHistory of Present Illness:\n40 yo M with IPF, s/p double lung tx [**2128**], h/o recurrent\npneumonia, chronic rejection and obliterative bronchiolitis,\npolymiositis presents with acute on chronic respiratory failure.\nWas sent from [**Hospital3 672**] Rehab for hypoxia (O2 sat 78%) and\nlethargy, also some report of migratory chest pain/pleuritic CP.\nVent settings prior to event AC 400/15/0.40/5, started to be\nambu bagged with improvement to 96%, then placed on FiO2 100%\nbut decreased again to 80%. At baseline can write and interact\nwith staff but was not doing so tod

The data looks messy and needs a lot of cleaning.